In [1]:
# https://platform.openai.com/docs/guides/image-generation?image-generation-model=gpt-image-1 
# https://lumalabs.ai/ray
# https://docs.lumalabs.ai/docs/python-video-generation 

In [3]:
# 이미지 준비 (gpt-img-1) model로 생성
# 내가 원하는 이미지 준비

from src.luma_generator import LumaGenerator
from src.img_generator import ImgGenerator

OPENAI_KEY = ""
LUMA_KEY = ""

AWS_ACCESS_KEY = ""
AWS_SECRET_KEY =  ""

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
luma_generator = LumaGenerator()
luma_generator.set_luma_client(LUMA_KEY)
luma_generator.set_s3_client(AWS_ACCESS_KEY, AWS_SECRET_KEY)

img_generator = ImgGenerator()
img_generator.set_openai_key(OPENAI_KEY)

In [6]:
# step1. 이미지 생성

prompt = """

케이뱅크에서 새로출시한 체크 카드 발급 유도를 위한 마케팅을 위한 9초짜리 쇼츠를 만드려고 해.
이 쇼츠는 서울 을지로 4가를 배경으로 20대 학생이 모델로 나오는 쇼츠로 기획하고 있어.

길을 걷고 있는 모습의 이미지였으면 좋겠어.

"""

images = ["data/card.png"]
imgs, prompt = img_generator.edit_multiple_images(
                                  prompt = prompt,
                                  images = images,
                                  n = 1,
                                  size = "1024x1024", 
                                  background = "opaque",
                                  quality = "auto",
                                  model = "gpt-image-1",
                                  timer_interval = 2
                                  )

이미지 1개 생성 중... (Elapsed: 48초)
[API 호출 완료] 총 소요 시간: 50.42초


이미지 변환 중: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 107.16img/s]


In [7]:
from PIL import Image
import io

image_bytes = imgs[0]
output_path = "sample.png"

image = Image.open(io.BytesIO(image_bytes))
image.save(output_path)

In [8]:
# step2. 비디오 생성을 위한 프롬프트 작성

from openai import OpenAI
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
        
luma_video_prompt = """
You are a cinematic scene describer trained to generate short, vivid prompts for Luma Labs Dream Machine.

Given a visual scene, generate a short cinematic prompt (15–35 words) that describes the subject, action, mood, lighting, and camera movement or angle.

Examples:

1. "A massive orb of water floating in a backlit forest"
2. "An explosion with camera shake"
3. "Raindrops in extreme slow motion"
4. "A snow leopard crouched on a rocky ledge, staring directly at camera, snowflakes falling around it"
5. "A detective in a dark trench coat and hat, holding a lantern, walks carefully down a narrow cobblestone alley, mist curling around his feet"
6. "At sunrise, place the camera at ground level, looking up at a solitary warrior standing on a rugged cliff edge, wind pulling at his cloak"
7. "A tiny chihuahua dressed in post-apocalyptic leathers with a WW2 bomber cap and goggles sits proudly on a makeshift throne"
8. "A cellist alone on stage under a single spotlight, camera slowly circling her as she begins to play"
9. "A gorilla surfing on a wave, water splashing dramatically as it leans into the motion"

Now, based on the input image or concept, generate a similarly styled prompt that is cinematic, physical, and emotionally engaging.

Output only the prompt text.

"""

client = OpenAI(api_key=OPENAI_KEY)

# Getting the Base64 string
base64_image = encode_image(output_path)

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": luma_video_prompt },
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{base64_image}",
                },
            ],
        }
    ],
)

print(response.output_text)

"Morning cityscape with a young man in a graphic sweatshirt and red cap, confidently striding forward. Camera at eye level, capturing bustling streets and distant skyscrapers bathed in soft urban light."


In [9]:
# step3. 비디오 생성

video_prompt = response.output_text
keyframes = [output_path]
video_bytes, prompt = luma_generator.generate_video(video_prompt,
                           keyframes=keyframes,
                           loop=True,
                           model="ray-2",
                           aspect_ratio="9:16",
                           resolution="720p",
                           duration="9s",
                        )

File uploaded successfully. Public URL: https://kbank-data-intelligence.s3.ap-northeast-2.amazonaws.com/kpick/img/sample.png
 비디오 생성 중... (Elapsed: 204초)

In [10]:
with open("shorts.mp4", "wb") as f:
    f.write(video_bytes)

In [11]:
# step4. 자막 생성

# Path to your image
image_path = "sample.png"

# Getting the Base64 string
base64_image = encode_image(image_path)

prompt = """

케이뱅크에서 새로나온 One체크카드를 소개하는 쇼츠 영상 자막 8줄 만들어줘

TTS로 말했을 때 총 발화 시간이 8.5초 이내여야 해.
각 줄은 되도록 8자 이내, 짧고 리듬감 있게 만들어줘.
말했을 때 자연스럽고 부드러운 흐름이 되도록 해줘.
번호나 따옴표 없이, 줄바꿈으로만 구분해줘.

쇼츠로 만드려고하는 이미지 썸네일도 같이 첨부 해줄게
반드시 추가 설명 없이 자막 8줄만 만들어줘.

카드 혜택 : 

✅ 케이뱅크 ONE 체크카드 (연회비·실적 조건 없음)
매월 원하는 캐시백 혜택을 자유롭게 선택할 수 있는 체크카드입니다.

1.여기서 더 캐시백

편의점, 카페, 배달앱, OTT, 영화 업종에서 7% 캐시백
건당 1만 원 이상 결제 시 적용 (일 1천 원, 월 2만5천 원 한도)

2. 모두 다 캐시백

오프라인 업종 0.7%, 온라인 쇼핑 1.2% 캐시백
실적 조건, 한도 없이 자동 적용

3. 369 캐시백

월 결제 횟수가 3회, 6회, 9회일 때 각 1,000원 캐시백
건당 5천 원 이상 결제 시 적용 (스탬프 형태로 확인 가능)

4. 대중교통 추가 캐시백

카드로 대중교통 월 5만 원 이상 이용 시 3,000원 추가 캐시백
"""


response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": prompt },
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{base64_image}",
                },
            ],
        }
    ],
)

print(response.output_text)

케이뱅크 카드
캐시백 쏙쏙!
편의점 OK
온라인 찰떡
대중교통 추가
스탬프로 확인
이제는 선택
혜택 듬뿍!


In [12]:
lines = response.output_text
lines = [line.strip() for line in lines.split("\n") if line.strip()]
print(lines)

['케이뱅크 카드', '캐시백 쏙쏙!', '편의점 OK', '온라인 찰떡', '대중교통 추가', '스탬프로 확인', '이제는 선택', '혜택 듬뿍!']


In [14]:
# step5. tts(text to speach)
# https://platform.openai.com/docs/guides/text-to-speech
import openai
import asyncio

import nest_asyncio
import asyncio

# jupyter notebook에서 아래 코드 실행
nest_asyncio.apply()

# 비동기 클라이언트 인스턴스 생성
async_client = openai.AsyncOpenAI(api_key=OPENAI_KEY)

# 개별 TTS 요청 (async)
async def generate_tts(client, text, index, voice="shimmer", model="tts-1"):
    filename = f"line_{index+1}.mp3"
    try:
        response = await async_client.audio.speech.create(
            model=model,
            voice=voice,
            input=text,
        )
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"✅ 생성 완료: {filename}")
        return filename
    except Exception as e:
        print(f"❌ TTS 오류 (line {index+1}): {e}")
        return None

# 여러 줄 비동기 처리
async def generate_all_tts(lines, voice="shimmer"):
    tasks = [
        generate_tts(client, line, idx, voice=voice)
        for idx, line in enumerate(lines)
    ]
    return await asyncio.gather(*tasks)

tts_files = asyncio.run(generate_all_tts(lines))

✅ 생성 완료: line_5.mp3
✅ 생성 완료: line_6.mp3
✅ 생성 완료: line_8.mp3
✅ 생성 완료: line_4.mp3
✅ 생성 완료: line_1.mp3
✅ 생성 완료: line_2.mp3
✅ 생성 완료: line_3.mp3
✅ 생성 완료: line_7.mp3


In [15]:
# step6. 오디오 파일 병합

import subprocess

#3. 오디오 파일 연결을 위한 concat list 파일 생성 ###
with open("concat_list.txt", "w") as f:
    for filename in tts_files:
        f.write(f"file '{filename}'\n")

# 4. TTS 오디오 병합 ###
print("🎵 TTS 오디오 병합 중...")
subprocess.run([
    "ffmpeg", "-y", "-f", "concat", "-safe", "0",
    "-i", "concat_list.txt", "-c", "copy", "combined.mp3"
], check=True)

🎵 TTS 오디오 병합 중...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

CompletedProcess(args=['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'concat_list.txt', '-c', 'copy', 'combined.mp3'], returncode=0)

In [16]:
# step7. 자막 생성 (정확히 1초 간격)
print("📝 SRT 자막 생성 중...")
with open("output.srt", "w", encoding="utf-8") as f:
    for i, line in enumerate(lines):
        start_time = f"00:00:0{i}.000"
        end_time = f"00:00:0{i+1}.000"
        f.write(f"{i+1}\n{start_time} --> {end_time}\n{line}\n\n")

📝 SRT 자막 생성 중...


In [18]:
# step8. 영상과 오디오, 자막 결합
video_input = "shorts.mp4"
output_video = "final_output.mp4"

subprocess.run([
    "ffmpeg", "-y",
    "-i", video_input,
    "-i", "combined.mp3",
    "-vf", "subtitles=output.srt",
    "-c:v", "libx264", "-c:a", "aac",
    output_video
], check=True)

print(f"✅ 완료: {output_video}")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

✅ 완료: final_output.mp4


[aac @ 0x144e64b50] Qavg: 1697.801
